# Matrix

In [37]:
import pandas as pd
import numpy as np

# Radiosensitivity rankings
sensitivity_rankings = {
    'Background': 1, 
    'Bone': 1, 
    'Obtur': 1,
    'TZ': 0,       
    'CG': 0,        
    'Bladder': 6,
    'SV': 5,        
    'Rectum': 9,   
    'NVB': 10,      
}

labels = list(sensitivity_rankings.keys())
values = list(sensitivity_rankings.values())
n_classes = len(labels)

# Create grids 
sens_of_predicted = np.array([values] * n_classes)
sens_of_actual    = np.array([values] * n_classes).T

# Calculate error gap
sens_matrix = sens_of_predicted - sens_of_actual

# Apply 2x penalty for false negatives, dangerous errors (predicted < actual)
FN_weighting = 2
sens_matrix[sens_matrix < 0] = sens_matrix[sens_matrix < 0] * FN_weighting
sens_matrix = np.abs(sens_matrix)

# COULD REPLACE MY APPROXIMATIONS WITH REAL VALUES BASED ON THE DATA INSTEAD
centroids = {
    'Background':(100, 100, 100),
    'TZ':        (0, 0, 0),       # Centre
    'CG':        (0, -5, 0),      # Touching TZ
    'Bladder':   (0, 10, 0),
    'Obtur':     (30, 0, 0),
    'Bone':      (50, 0, 0),      # Far laterally
    'Rectum':    (0, -15, 0),     # Posterior to prostate
    'SV':        (0, 5, 5),       # Superior/Posterior
    'NVB':       (5, -10, 0),     # Postero-lateral
}

# Euclidean distances
dist_matrix = np.zeros((n_classes, n_classes))
coords = [centroids[label] for label in labels]

for i in range(n_classes):
    for j in range(n_classes):
        # Distance between centroids
        dist = np.linalg.norm(np.array(coords[i]) - np.array(coords[j]))
        dist_matrix[i, j] = dist

# Invert distance for proximity weight
proximity_matrix = 100 / (dist_matrix + 10.0) 

np.fill_diagonal(proximity_matrix, 0)

total_weight_matrix = sens_matrix + proximity_matrix

df = pd.DataFrame(np.round(total_weight_matrix, 1), index=labels, columns=labels)

#Normalize matrix weights, average 1
df = (df * 81 / df.sum().sum())

# New stuff @Sami

In [38]:
# Placeholder of distance matrix:

distance_matrix = np.ones((9, 9))
distance_matrix[7] = 1.5 # Rectum is typically close to Prostate => Rectum row i.e. where the "Actual" is Rectum, gets a higher distance weighting
distance_matrix[8] = 2 #NVB very close
distance_matrix[1] = 0.5 # Bone very far

In [39]:
accuracy_matrix = 1 - np.eye(9) # Used for asserting that we want segmentations to be accurate in general, to give doctors a better picture of the overall image

In [40]:
print(sens_matrix)
print('-' * 50)
print(distance_matrix)
print('-' * 50)
print(accuracy_matrix)

[[ 0  0  0  2  2  5  4  8  9]
 [ 0  0  0  2  2  5  4  8  9]
 [ 0  0  0  2  2  5  4  8  9]
 [ 1  1  1  0  0  6  5  9 10]
 [ 1  1  1  0  0  6  5  9 10]
 [10 10 10 12 12  0  2  3  4]
 [ 8  8  8 10 10  1  0  4  5]
 [16 16 16 18 18  6  8  0  1]
 [18 18 18 20 20  8 10  2  0]]
--------------------------------------------------
[[1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5 0.5]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.  1.  1.  1.  1.  1.  1.  1.  1. ]
 [1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5 1.5]
 [2.  2.  2.  2.  2.  2.  2.  2.  2. ]]
--------------------------------------------------
[[0. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 0. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 0. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0.]]


In [41]:
sens_matrix.sum()

np.int64(486)

In [42]:
# Final matrix calculation:

# Normalize
sens_matrix = sens_matrix * 72 / sens_matrix.sum() # Now both sens and acc matrices have a total of 72

sens_importance = 0.75

# Calculate overall importance
importance_matrix = (sens_importance * sens_matrix + (1 - sens_importance) * accuracy_matrix)

# Apply distances
overall_matrix = importance_matrix * distance_matrix # We multiply both the accuracy and sensitivity matrices by distance, since both concerns are more important closer to the region of interest i.e. prostate

# Normalize a final time
final_matrix = overall_matrix * 72 / overall_matrix.sum()

df = pd.DataFrame(np.round(final_matrix, 2), index=labels, columns=labels)

df

,Background,Bone,Obtur,TZ,CG,Bladder,SV,Rectum,NVB
Background,0.00,0.20,0.20,0.38,0.38,0.64,0.55,0.91,0.99
Bone,0.10,0.00,0.10,0.19,0.19,0.32,0.28,0.45,0.50
Obtur,0.20,0.20,0.00,0.38,0.38,0.64,0.55,0.91,0.99
TZ,0.29,0.29,0.29,0.00,0.20,0.73,0.64,0.99,1.08
CG,0.29,0.29,0.29,0.20,0.00,0.73,0.64,0.99,1.08
Bladder,1.08,1.08,1.08,1.26,1.26,0.00,0.38,0.46,0.55
SV,0.91,0.91,0.91,1.08,1.08,0.29,0.00,0.55,0.64
Rectum,2.42,2.42,2.42,2.69,2.69,1.09,1.36,0.00,0.43
NVB,3.58,3.58,3.58,3.93,3.93,1.81,2.17,0.75,0.00


In [43]:
df.sum().sum()

np.float64(72.02000000000001)

## Metrics and functions

In [ ]:
import torch

weight_matrix = torch.Tensor([
    [0, 2, 1],
    [1, 0, 1],
    [1, 1, 0],
])

target = torch.Tensor([[
    [1, 1, 1, 0],
    [1, 1, 1, 0],
    [1, 1, 1, 0],
    [2, 2, 0, 0],
]])

pred = torch.Tensor([[
    [0, 1, 1, 1],
    [0, 1, 1, 1],
    [2, 1, 1, 1],
    [2, 2, 0, 0],
]])

def compute_weighted_dice_score_fast(pred, target, weight_matrix, num_classes=3, epsilon=1e-6):

    pred = pred.view(-1).long()
    target = target.view(-1).long()

    ids = target * num_classes + pred
    
    cm = torch.bincount(ids, minlength=num_classes**2).view(num_classes, num_classes).t().float()

    weighted_cm = cm * weight_matrix

    tp = cm.diagonal()

    fp = weighted_cm.sum(dim=1) # Summing across rows, where we always predict a particular class

    fn = weighted_cm.sum(dim=0) # Summing across columns, where a particular class is always the target

    score = (2 * tp) / (2 * tp + fp + fn + epsilon)

    return score

scores = compute_weighted_dice_score_fast(pred, target, weight_matrix)

print("Confusion Matrix (Internal Calculation):")

print("-" * 30)
print(f"Class 0 Loss: {scores[0]:.4f}")
print(f"Class 1 Loss: {scores[1]:.4f}")
print(f"Class 2 Loss: {scores[2]:.4f}")
print("-" * 30)
print(f"Mean Dice Loss: {scores.mean():.4f}")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class WeightedDiceScore(nn.Module):
    def __init__(self, weight_matrix, epsilon=1e-6):
        super().__init__()
        self.register_buffer("weight_matrix", weight_matrix)
        self.epsilon = epsilon

    def forward(self, pred, target):
        """
        Args:
            pred (torch.Tensor): Probabilities (B, C, H, W) - Output of Softmax
            target (torch.Tensor): Ground Truth Indices (B, H, W)
        """
        num_classes = pred.shape[1]
        
        # 1. Convert Target to One-Hot (B, C, H, W)
        # We assume target has shape (B, H, W)
        target_onehot = F.one_hot(target, num_classes).permute(0, 3, 1, 2).float()

        # 2. Flatten spatial dimensions for efficient matrix multiplication
        # Shapes become: (B, C, N) where N = H*W
        pred_flat = pred.flatten(2) 
        target_flat = target_onehot.flatten(2)
        
        # 3. Compute Soft Confusion Matrix via Einsum
        # Equation: For every batch b, sum over pixels n: pred(c) * target(k)
        # Result: (C, C) matrix where row=Pred, col=Target
        # This effectively sums up the "probability mass" for every pred/target pair
        soft_cm = torch.einsum("bcn, bkn -> ck", pred_flat, target_flat)
        
        # 4. Apply Weights
        # weight_matrix is (C, C). Element-wise multiplication applies penalties.
        # Since diagonal of weight_matrix is 0, TP contributions are zeroed out here.
        weighted_cm = soft_cm * self.weight_matrix

        # 5. Calculate Components
        
        # TP: Diagonal of the original Soft Confusion Matrix (unweighted)
        tp = torch.diagonal(soft_cm)
        
        # FP_weighted: Sum of Weighted Rows (Predicted c, Actual k)
        # sum(dim=1) collapses columns
        fp_weighted = weighted_cm.sum(dim=1)
        
        # FN_weighted: Sum of Weighted Columns (Actual c, Predicted k)
        # sum(dim=0) collapses rows
        fn_weighted = weighted_cm.sum(dim=0)

        # 6. Dice Formula
        numerator = 2 * tp
        denominator = (2 * tp) + fp_weighted + fn_weighted + self.epsilon
        
        scores = numerator / denominator
        
        return scores

In [ ]:
weight_matrix = torch.Tensor([
    [0, 1, 1],
    [1, 0, 1],
    [1, 1, 0],
])

# Create a differentiable "Model Output" (Logits)
# Batch=1, Classes=3, Height=2, Width=2
# logits = torch.randn(1, 3, 2, 2, requires_grad=True)
a = 1000.0
logits = torch.tensor([[
    [[a, 0.0],
    [0.0, 0.0]],
    [[0.0, a],
    [a, a]],
    [[0.0, 0.0],
    [0.0, 0.0]],
]], requires_grad=True)

# Apply Softmax to get probabilities (Required for this loss)
probs = F.softmax(logits, dim=1)

# Target (Indices)
target = torch.tensor([[[0, 1],
                        [1, 2]]]) # Shape (1, 2, 2)

# 2. Initialize Module
dice_calc = WeightedDiceScore(weight_matrix)

# 3. Forward Pass
scores = dice_calc(probs, target)
print(f"Dice Scores per class: {scores}")

# 4. Backward Pass (Proof of differentiation)
# We want to maximize Dice, so we minimize (1 - Dice) or (-Dice)
loss = 1 - scores.mean()
loss.backward()

print("\nGradients exist and are computed:", logits.grad is not None)
print(f"Gradient shape: {logits.grad.shape}")